### **Paso 1:**
Primero conectamos nuestra cuenta de Drive con el fin de subir los  datasets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Paso 2:**
Cargamos el dataset que contiene las enfermedades con sus respectivos sintomas, son un total de 263 síntomas únicos, 201 enfermedades, 263 columnas y 50.250 filas.

In [2]:
import pandas as pd

file_path = '/content/drive/MyDrive/PI_2024-2/Datasets/medical_dataset.csv'

# Cargar el dataset
df = pd.read_csv(file_path)

print(df.head())

         diseases  sharp abdominal pain  vomiting  cough  headache  back pain  \
0  panic disorder                     0         0      0         0          0   
1  panic disorder                     0         0      0         0          0   
2  panic disorder                     0         0      0         0          0   
3  panic disorder                     0         0      0         0          0   
4  panic disorder                     0         0      0         0          0   

   nausea  fever  sharp chest pain  shortness of breath  ...  \
0       0      0                 0                    1  ...   
1       0      0                 0                    1  ...   
2       0      0                 0                    1  ...   
3       0      0                 0                    0  ...   
4       0      0                 0                    0  ...   

   knee stiffness or tightness  recent pregnancy  hip stiffness or tightness  \
0                            0                 0

### **Paso 3:**
Dividimos nuestro dataframe en 'X' y 'y', siendo 'X' las características y 'y' las etiquetas

In [3]:
# Separar características y etiquetas
X = df.drop(columns=['diseases'])
y = df['diseases']


# **Enfoque 1**
#### Haremos reducción de dimensionalidad aplicando SVD, usaremos ajuste de hiper parametros con Random Search y validación cruzada, entrenando 3 modelos diferentes:
- #### Regresión Logística Multinomial
- #### RandomForest
- #### XGBoost

### **Paso 1:**
Nuestros datos son dispersos, ya que tenemos 263 columnas que representan los síntomas, pero cada enfermedad solo tiene unos pocos síntomas asociados, lo que genera una gran cantidad de '0'. Para solucionar esto, aplicaremos reducción de dimensionalidad utilizando Descomposición en Valores Singulares (SVD), lo que nos permitirá reducir la complejidad del modelo y entrenarlo más rápidamente.

In [4]:
from sklearn.decomposition import TruncatedSVD

# Aplicar SVD para reducción dimensional
svd = TruncatedSVD(n_components=100, random_state=42)  # Reduce a 100 dimensiones
X_reduced = svd.fit_transform(X)

print("Dimensiones originales:", X.shape)
print("Dimensiones reducidas:", X_reduced.shape)

Dimensiones originales: (50250, 263)
Dimensiones reducidas: (50250, 100)


### **Paso 2:**
Dividir los datos en entrenamiento y prueba

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42, stratify=y)

print("Tamaño de entrenamiento:", X_train.shape)
print("Tamaño de prueba:", X_test.shape)

Tamaño de entrenamiento: (40200, 100)
Tamaño de prueba: (10050, 100)


### **Paso 3:**
Entramos nuestro modelo con los datos que acabamos de dividir, aplicando también ajuste de hiper parametros y validación cruzada

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import numpy as np
import time
import joblib
import os

# Rutas para guardar modelos
models_dir = "/content/drive/MyDrive/PI_2024-2/modelos_entrenados/"
os.makedirs(models_dir, exist_ok=True)

logreg_path = os.path.join(models_dir, "logreg_model.pkl")
rf_path = os.path.join(models_dir, "rf_model.pkl")
xgb_path = os.path.join(models_dir, "xgb_model.pkl")

# Codificar etiquetas de las enfermedades
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Verificar si los modelos ya están guardados
if os.path.exists(logreg_path) and os.path.exists(rf_path) and os.path.exists(xgb_path):
    # Cargar modelos guardados
    print("Cargando modelos entrenados desde Google Drive...")
    best_logreg_model = joblib.load(logreg_path)
    best_rf_model = joblib.load(rf_path)
    best_xgb_model = joblib.load(xgb_path)
else:
    # 1. Regresión Logística - Ajuste de hiperparámetros
    logreg_param_grid = {
        'C': np.logspace(-4, 4, 50),
        'solver': ['lbfgs', 'saga'],
        'multi_class': ['multinomial']
    }
    logreg_model = LogisticRegression(max_iter=1000, random_state=42)
    logreg_search = RandomizedSearchCV(
        logreg_model, logreg_param_grid, n_iter=5, cv=3, scoring='accuracy', random_state=42
    )

    start_time = time.time()
    logreg_search.fit(X_train, y_train)
    logreg_train_time = time.time() - start_time
    best_logreg_model = logreg_search.best_estimator_
    joblib.dump(best_logreg_model, logreg_path)  # Guardar modelo
    print("Modelo de Regresión Logística guardado.")

    # 2. Random Forest - Ajuste de hiperparámetros
    rf_param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }
    rf_model = RandomForestClassifier(random_state=42)
    rf_search = RandomizedSearchCV(
        rf_model, rf_param_grid, n_iter=5, cv=2, scoring='accuracy', random_state=42
    )

    start_time = time.time()
    rf_search.fit(X_train, y_train)
    rf_train_time = time.time() - start_time
    best_rf_model = rf_search.best_estimator_
    joblib.dump(best_rf_model, rf_path)  # Guardar modelo
    print("Modelo de Random Forest guardado.")

    # 3. XGBoost - Ajuste de hiperparámetros
    xgb_param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2, 0.3],
        'max_depth': [3, 6, 10],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'gamma': [0, 1, 5],
    }
    xgb_model = xgb.XGBClassifier(eval_metric='mlogloss', random_state=42)
    xgb_search = RandomizedSearchCV(
        xgb_model, xgb_param_grid, n_iter=3, cv=2, scoring='accuracy', random_state=42
    )

    start_time = time.time()
    xgb_search.fit(X_train, y_train_encoded)
    xgb_train_time = time.time() - start_time
    best_xgb_model = xgb_search.best_estimator_
    joblib.dump(best_xgb_model, xgb_path)  # Guardar modelo
    print("Modelo de XGBoost guardado.")

# Predicciones con los modelos cargados o entrenados
logreg_y_pred = best_logreg_model.predict(X_test)
rf_y_pred = best_rf_model.predict(X_test)
xgb_y_pred = best_xgb_model.predict(X_test)

# Evaluación de los modelos
print("\nEvaluación de los modelos:")
# Regresión Logística
logreg_accuracy = accuracy_score(y_test, logreg_y_pred)
print(f"\nRegresión Logística - Accuracy: {logreg_accuracy:.4f}")
print("Mejores hiperparámetros:", best_logreg_model.get_params())
print("\nClassification Report (Logistic Regression):")
print(classification_report(y_test, logreg_y_pred))

# Random Forest
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f"\nRandom Forest - Accuracy: {rf_accuracy:.4f}")
print("Mejores hiperparámetros:", best_rf_model.get_params())
print("\nClassification Report (Random Forest):")
print(classification_report(y_test, rf_y_pred))


Cargando modelos entrenados desde Google Drive...

Evaluación de los modelos:

Regresión Logística - Accuracy: 0.8706
Mejores hiperparámetros: {'C': 494.1713361323828, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'multinomial', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

Classification Report (Logistic Regression):
                                                 precision    recall  f1-score   support

                              actinic keratosis       0.90      0.74      0.81        50
                            acute bronchiolitis       0.94      0.92      0.93        50
                               acute bronchitis       0.84      0.52      0.64        50
                             acute bronchospasm       0.68      0.86      0.76        50
                            acute kidney injury       0.94      0.94 

In [7]:
# XGBoost
xgb_accuracy = accuracy_score(y_test_encoded, xgb_y_pred)
print(f"\nXGBoost - Accuracy: {xgb_accuracy:.4f}")
print("Mejores hiperparámetros:", best_xgb_model.get_params())
classes_in_test_and_pred = set(y_test_encoded) & set(xgb_y_pred)
print("\nClassification Report (XGBoost):")
print(classification_report(
    y_test_encoded, xgb_y_pred, labels=list(classes_in_test_and_pred),
    target_names=label_encoder.classes_[list(classes_in_test_and_pred)]
))


XGBoost - Accuracy: 0.8043
Mejores hiperparámetros: {'objective': 'multi:softprob', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.6, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'mlogloss', 'feature_types': None, 'gamma': 0, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.3, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 10, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.6, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}

Classification Report (XGBoost):
                   